In [1]:
import numpy as np
from tensorflow.keras.datasets import mnist
import tensorflow as tf
from tensorflow.keras import Model, layers
import pandas as pd

num_classes = 0
def load_data():
    global num_classes
    data = pd.read_csv('pump.csv')
    num_classes = data['type'].value_counts().shape[0]
    train_data = data.sample(frac=0.9,random_state=None)
    test_data = data.drop(train_data.index)
    return train_data.values,test_data.values

def parse_data():
    train_data,test_data = load_data()
    n = train_data.shape[1]-1
    x_train = train_data[:,0:n]
    y_train = train_data[:,n]
    n = test_data.shape[1]-1
    x_test = test_data[:,0:n]
    y_test = test_data[:,n]
    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = parse_data()
#往往输入是文件形式，此处api已经封装好了，直接取出了训练集和验证集
#均用浮点运算
#x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

n_hidden_1 = 1024 # 1st layer number of neurons.
n_hidden_2 = 512  # 2nd layer number of neurons.
n_hidden_3 = 256 # 2nd layer number of neurons.
#num_classes = x_train.shape[1] * x_train.shape[2]
learning_rate = 0.001

#x_train = x_train.values().reshape(-1,num_classes)
#x_test = x_test.values().reshape(-1,num_classes)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#train_data = train_data.repeat().batch(256).prefetch(1)
train_data = train_data.repeat().shuffle(5000).batch(256).prefetch(1)

class NeuralNet(Model):
    # Set layers.
    def __init__(self):
        super(NeuralNet, self).__init__()
        # First fully-connected hidden layer.
        self.fc1 = layers.Dense(n_hidden_1, activation=tf.nn.relu)
        # First fully-connected hidden layer.
        self.fc2 = layers.Dense(n_hidden_2, activation=tf.nn.relu)
        self.fc3 = layers.Dense(n_hidden_3, activation=tf.nn.relu)
        # Second fully-connecter hidden layer.
        self.out = layers.Dense(num_classes)

    # Set forward pass.
    def call(self, x, is_training=False):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            x = tf.nn.softmax(x)
        return x

# Build neural network model.
neural_net = NeuralNet()

def cross_entropy_loss(x, y):
    # Convert labels to int 64 for tf cross-entropy function.
    y = tf.cast(y, tf.int64)
    # Apply softmax to logits and compute cross-entropy.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    # Average loss across the batch.
    return tf.reduce_mean(loss)

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

optimizer = tf.optimizers.SGD(learning_rate)



In [2]:
for i, (x, y) in enumerate(train_data.take(10000), 1):
    with tf.GradientTape() as g:
        pred = neural_net(x, is_training=True)
        loss = cross_entropy_loss(pred, y)
    trainable_variables = neural_net.trainable_variables
    gradients = g.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))  

    if i % 100 == 0:
        pred = neural_net(x, is_training=True)
        loss = cross_entropy_loss(pred, y)
        acc = accuracy(pred, y)
        print("step: %i, loss: %f, accuracy: %f" % (i, loss, acc))

pred = neural_net(x_test)
loss = cross_entropy_loss(pred, y_test)
acc = accuracy(pred, y_test)
print("loss: %f, accuracy: %f" % ( loss, acc))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

step: 100, loss: 1.348744, accuracy: 0.386719
step: 200, loss: 1.317535, accuracy: 0.378906
step: 300, loss: 1.307368, accuracy: 0.375000
step: 400, loss: 1.285458, accuracy: 0.367188
step: 500, loss: 1.273631, accuracy: 0.292969
step: 600, loss: 1.252223, accuracy: 0.410156
step: 700, loss: 1.247421, accuracy: 0.375000
step: 800, loss: 1.232376, accuracy: 0.363281
step: 900, loss: 1.239834, accuracy: 0.394531
step: 1000, loss: 1.233997, accuracy: 0.367188
step: 1100, loss: 1.234756, accuracy: 0.414062


KeyboardInterrupt: 

Error: Pip module debugpy is required for debugging cells. You will need to install it to debug cells.